# N-gram Language Model: Auto-Complete System

This project builds an auto-complete system using N-gram language models. N-grams assign probabilities to word sequences, enabling prediction of the next most likely word given a context.

**Steps:**
1. Load and preprocess data (tokenization, train/test split, OOV handling with `<unk>`)
2. Build N-gram language models with k-smoothing
3. Evaluate with perplexity
4. Implement auto-complete suggestions

## Part 1: Load and Preprocess Data

In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

In [2]:
with open("./en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")
print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 164456396
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


'od. I see the success you got poppin in yo area.\nRT : Consumers are visual. They want data at their finger tips. Mobile is the only way to deliver this, 24/7.\nu welcome\nIt is #RHONJ time!!\nThe key to keeping your woman happy= attention, affection, treat her like a queen and sex her like a pornstar!\n'

-------


### Split data into sentences

In [3]:
def split_to_sentences(data):
    sentences = []
    [sentences.append(line.strip()) for line in data.split("\n") if line.strip()]
    return sentences

In [4]:
x = "\nI have a pen.\nI have an apple. \nAh\nApple pen.\n\n"
print(split_to_sentences(x))

['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']


### Tokenize sentences
Split each sentence into tokens and convert to lowercase.

In [5]:
from nltk.tokenize import WordPunctTokenizer

def tokenize_sentences(sentences):
    tokenized_sentences = []
    tk = WordPunctTokenizer()
    for word in sentences:
        word = word.lower()
        word_token = tk.tokenize(word)
        tokenized_sentences.append(word_token)
    return tokenized_sentences

In [6]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

### Get tokenized data and split into train/test

In [7]:
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [8]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

2360148 data are split into 1888118 train and 472030 test set


### Handle Out-of-Vocabulary (OOV) words
Words appearing fewer than N times in training are replaced with `<unk>`.

In [9]:
def count_words(tokenized_sentences):
    word_counts = {}
    for sentence in tokenized_sentences:
        for word in sentence:
            word_counts[word] = word_counts.get(word, 0) + 1
    return word_counts

In [10]:
def get_words_with_nplus_frequency(tokenized_sentences, minimum_freq):
    filtered_words = []
    word_counts = count_words(tokenized_sentences)
    for each in word_counts:
        if word_counts[each] >= minimum_freq:
            filtered_words.append(each)
    return filtered_words

In [11]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_marker="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for word in sentence:
            if word not in vocabulary:
                replaced_sentence.append(unknown_marker)
            else:
                replaced_sentence.append(word)
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

In [12]:
def preprocess_data(train_data, test_data, minimum_freq):
    vocabulary = get_words_with_nplus_frequency(train_data, minimum_freq=minimum_freq)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

In [13]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(
    train_data, test_data, minimum_freq)

print("First preprocessed training sample:")
print(train_data_processed[0])
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['adam', 'sandler', 'is', 'not', 'being', 'funny', 'on', '#', '<unk>', '.', 'make', 'it', 'stop']
Size of vocabulary: 129479


## Part 2: N-gram Language Models

The conditional probability of the next word given the previous n words:

$$ P(w_t | w_{t-1}\dots w_{t-n}) $$

Estimated with k-smoothing:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} $$

where $|V|$ is the vocabulary size. For unseen n-grams, this reduces to $1/|V|$.

### Count N-grams

In [14]:
def count_n_grams(data, n):
    n_grams = {}
    for sentence in data:
        sentence = ["<s>"]*n + sentence + ["<e>"]
        sentence = tuple(sentence)
        for i in range(len(sentence) - n + 1):
            n_gram_key = tuple(sentence[i: i+n])
            n_grams[n_gram_key] = n_grams.get(n_gram_key, 0) + 1
    return n_grams

In [15]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


### Estimate probability with k-smoothing

In [16]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    new_gram = previous_n_gram + (word,)
    probability = (n_plus1_gram_counts.get(new_gram, 0) + k) / (n_gram_counts.get(previous_n_gram, 0) + (k * vocabulary_size))
    return probability

In [17]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    return probabilities

In [18]:
# Test: P(cat | a) with bigram model
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
print(estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1))

0.3333333333333333


### Count and Probability Matrices

In [19]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [20]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
display(make_count_matrix(bigram_counts, unique_words))
display(make_probability_matrix(bigram_counts, unique_words, k=1))

,this,cat,i,is,dog,a,like,<e>,<unk>
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(i,)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(like,)",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(is,)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(<s>,)",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(dog,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


,this,cat,i,is,dog,a,like,<e>,<unk>
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(i,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(like,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909
"(is,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(<s>,)",0.181818,0.090909,0.181818,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(a,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(dog,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000


## Part 3: Perplexity

Perplexity evaluates how well the language model predicts a test set:

$$ PP(W) = \sqrt[N]{\prod_{t=1}^N \frac{1}{P(w_t | w_{t-1})}} $$

Lower perplexity = better model.

In [21]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    n = len(list(n_gram_counts.keys())[0])
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    N = len(sentence)
    p = 1.0
    for i in range(N - n):
        previous_n_gram = sentence[i : i + n]
        word = sentence[i + n]
        probability = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size)
        p *= probability
    perplexity = (1/p)**(1/N)
    return perplexity

In [22]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

perplexity_train1 = calculate_perplexity(sentences[0], unigram_counts, bigram_counts, len(unique_words), k=1.0)
print("Perplexity for first train sample:", perplexity_train1)

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence, unigram_counts, bigram_counts, len(unique_words), k=1.0)
print("Perplexity for test sample:", perplexity_test)

Perplexity for first train sample: 2.8039657955522013
Perplexity for test sample: 3.965406456500188


## Part 4: Auto-Complete System

Compute probabilities for all possible next words and suggest the most likely one. Optionally filter by a prefix (`start_with`).

In [23]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    n = len(list(n_gram_counts.keys())[0])
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)
    suggestion = None
    max_prob = 0
    for next_word in probabilities:
        if start_with is None or next_word.startswith(start_with):
            if probabilities[next_word] > max_prob:
                max_prob = probabilities[next_word]
                suggestion = next_word
    return suggestion, max_prob

In [24]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [25]:
# Test on toy data
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
print(suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0))
print(suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with="c"))

('a', 0.2727272727272727)
('cat', 0.09090909090909091)


## Running on Full Twitter Dataset

In [26]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [27]:
previous_tokens = ["i", "am", "to"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('be', 0.049358192117169596),
 ('see', 0.00013871335655497673),
 ('have', 9.259687948516135e-05),
 ('adam', 7.723140846919624e-06)]

In [28]:
previous_tokens = ["i", "want", "to", "go"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('to', 0.05924903985980089),
 ('to', 0.02446685517683275),
 ('to', 0.004293911355943859),
 ('to', 0.00171523947973633)]

In [29]:
previous_tokens = ["hey", "how", "are"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[('you', 0.07218642243060648),
 ('you', 0.019281549746097422),
 ('you', 0.0005479578921371901),
 ('adam', 7.723140846919624e-06)]

In [30]:
previous_tokens = ["hey", "how", "are", "you"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

[("'", 0.06830682592961662),
 ('?', 0.011561904504480153),
 ('?', 0.007315798641351681),
 ('?', 0.00016209832421208636)]

In [31]:
previous_tokens = ["hey", "how", "are", "you"]
get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

[('don', 0.010340181989825352),
 ('doing', 0.00762910005000473),
 ('doing', 0.0020144952780533614),
 ('doing', 6.175174255698529e-05)]